# Telecom

## Import Packages

In [42]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib as mpl
import seaborn as sns
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, log_loss

from sklearn.impute import SimpleImputer
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neural_network import MLPClassifier
import statsmodels.formula.api as smf

from patsy import dmatrices, dmatrix, build_design_matrices

import math

In [2]:
# This sets some nicer defaults for plotting.
# This must be run in a separate cell from importing matplotlib due to a bug.
params = {'legend.fontsize': 'large',
          'figure.figsize': (11.0, 11.0),
          'axes.labelsize': 'x-large',
          'axes.titlesize':'xx-large',
          'xtick.labelsize':'large',
          'ytick.labelsize':'large'}
mpl.rcParams.update(params)

# This makes it so that the pandas dataframes don't get truncated horizontally.
pd.options.display.max_columns = 200

## Load and Inspect the Data

In [4]:
telecom_df = pd.read_csv("WA_Fn-UseC_-Telco-Customer-Churn.csv")

What are the columns of the dataset?

In [5]:
telecom_df.columns

Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')

How many rows and columns do we have?

In [6]:
telecom_df.shape

(7043, 21)

In [7]:
telecom_df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [8]:
def summarize_dataframe(df):
    missing_values = pd.concat([pd.DataFrame(df.columns, columns=['Variable Name']), 
                      pd.DataFrame(df.dtypes.values.reshape([-1,1]), columns=['Data Type']),
                      pd.DataFrame(df.isnull().sum().values, columns=['Missing Values']), 
                      pd.DataFrame([df[name].nunique() for name in df.columns], columns=['Unique Values'])], 
                     axis=1).set_index('Variable Name')
    return pd.concat([missing_values, df.describe(include='all').transpose()], axis=1).fillna("")

What are the names of the variables in the data frame? How many values are missing? Use our self defined `summarize_dataframe` function to find out the number of NA's for each variable.

In [9]:
summarize_dataframe(telecom_df)

/tmp/ipykernel_739/2366052593.py:7: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  return pd.concat([missing_values, df.describe(include='all').transpose()], axis=1).fillna("")


,Data Type,Missing Values,Unique Values,count,unique,top,freq,mean,std,min,25%,50%,75%,max
customerID,object,0,7043,7043.0,7043,3186-AJIEK,1,,,,,,,
gender,object,0,2,7043.0,2,Male,3555,,,,,,,
SeniorCitizen,int64,0,2,7043.0,,,,0.162147,0.368612,0.0,0.0,0.0,0.0,1.0
Partner,object,0,2,7043.0,2,No,3641,,,,,,,
Dependents,object,0,2,7043.0,2,No,4933,,,,,,,
tenure,int64,0,73,7043.0,,,,32.371149,24.559481,0.0,9.0,29.0,55.0,72.0
PhoneService,object,0,2,7043.0,2,Yes,6361,,,,,,,
MultipleLines,object,0,3,7043.0,3,No,3390,,,,,,,
InternetService,object,0,3,7043.0,3,Fiber optic,3096,,,,,,,
OnlineSecurity,object,0,3,7043.0,3,No,3498,,,,,,,


## Engineer Features

In [28]:
telecom_df["Churn_Num"] = np.where(telecom_df['Churn']=='Yes', 1, 0)

## Creating Training and Testing Sets

Once we have this, we can take all of the data before February 19, 2008 as our training set, and all of the data after that point as our testing set.

In [29]:
telecom_df_train, telecom_df_test = train_test_split(telecom_df, test_size=0.2)

## Preprocess Training and Testing Data

In [30]:
list(telecom_df_train.columns)

['customerID',
 'gender',
 'SeniorCitizen',
 'Partner',
 'Dependents',
 'tenure',
 'PhoneService',
 'MultipleLines',
 'InternetService',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingTV',
 'StreamingMovies',
 'Contract',
 'PaperlessBilling',
 'PaymentMethod',
 'MonthlyCharges',
 'TotalCharges',
 'Churn',
 'Churn_Num']

From the list above, choose some independent variables to put into your models.

In [31]:
ind_var_selected = ['tenure','MultipleLines','Dependents','SeniorCitizen']

In [32]:
X_train = telecom_df_train[ind_var_selected]
y_train = telecom_df_train['Churn']

In [33]:
X_test = telecom_df_test[ind_var_selected]
y_test = telecom_df_test['Churn']

## Scoring Rules and Benchmarks

In [34]:
bench_5050 = np.repeat(0.5, len(telecom_df_test))

In [35]:
log_loss(y_test, bench_5050)

0.6931471805599454

## Fit a Logistic Regression

In [37]:
lr = smf.logit(formula='Q("Churn_Num") ~ Q("tenure") + Q("MultipleLines") + Q("Dependents") + Q("SeniorCitizen")', data=telecom_df_train).fit()
lr.summary()

Optimization terminated successfully.
         Current function value: 0.476253
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:         Q("Churn_Num")   No. Observations:                 5634
Model:                          Logit   Df Residuals:                     5628
Method:                           MLE   Df Model:                            5
Date:                Mon, 16 Sep 2024   Pseudo R-squ.:                  0.1745
Time:                        02:22:02   Log-Likelihood:                -2683.2
converged:                       True   LL-Null:                       -3250.5
Covariance Type:            nonrobust   LLR p-value:                4.212e-243
==========================================================================================================
                                             coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------
Intercept                                 -0.2191      0.058     -3.779      0.000      -0.333      -0.105
Q("MultipleLines")[T.No phone service]     0.3544      0.120      2.965      0.003       0.120       0.589
Q("MultipleLines")[T.Yes]                  1.0242      0.079     12.974      0.000       0.869       1.179
Q("Dependents")[T.Yes]                    -0.5956      0.085     -7.035      0.000      -0.761      -0.430
Q("tenure")                               -0.0471      0.002    -25.985      0.000      -0.051      -0.044
Q("SeniorCitizen")                         0.7160      0.088      8.182      0.000       0.545       0.888
==========================================================================================================
"""

In [38]:
lr_pred = lr.predict(telecom_df_test)

In [39]:
log_loss(y_test, lr_pred)

0.4960295403832431

In [40]:
lr.params

Intercept                                -0.219060
Q("MultipleLines")[T.No phone service]    0.354421
Q("MultipleLines")[T.Yes]                 1.024205
Q("Dependents")[T.Yes]                   -0.595570
Q("tenure")                              -0.047116
Q("SeniorCitizen")                        0.716009
dtype: float64